In [3]:
import pandas as pd
import io
from minio import Minio
import os

PATH = "ERP/7eData/2022/"
FILE_NAME = "recintos_almendros_parcelas.xlsx"


ACCESS_ROOT = os.environ.get("PREFECT_MINIO_ACCESS_ROOT")
SECRET_ROOT = os.environ.get("PREFECT_MINIO_SECRET_ROOT")
MINIO_HOST = os.environ.get("PREFECT_MINIO_HOST")
minio_client = Minio(MINIO_HOST, access_key=ACCESS_ROOT,
                     secret_key=SECRET_ROOT, secure=False)

# Get pistacho.json from MinIO and deserialize it
data = minio_client.get_object(
    "trusted-zone",PATH + FILE_NAME).read()
df = pd.read_excel(io.BytesIO(data), engine="openpyxl",
                    sheet_name="Sheet1", na_values=[""])
len(df)

S3Error: S3 operation failed; code: NoSuchKey, message: The specified key does not exist., resource: /trusted-zone/ERP/7eData/2022recintos_almendros_parcelas.xlsx, request_id: 1750DAE500809D16, host_id: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855, bucket_name: trusted-zone, object_name: ERP/7eData/2022recintos_almendros_parcelas.xlsx